<a href="https://colab.research.google.com/github/cgberlin/stylegan2-ada/blob/main/create_music_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
All the random imports we need, can collapse and just click run
"""
import os
from IPython.display import Audio

**Clone the latest version of my forked StyleGAN2-Ada**

In [4]:
%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/cgberlin/stylegan2-ada.git
%cd stylegan2-ada

print('Tensorflow version: {}'.format(tf.__version__) )
!nvidia-smi -L
print('GPU Identified at: {}'.format(tf.test.gpu_device_name()))

TensorFlow 1.x selected.
fatal: destination path 'stylegan2-ada' already exists and is not an empty directory.
/content/stylegan2-ada
Tensorflow version: 1.15.2
GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-2ea1cd6c-af80-1565-edec-74dc0d34ff34)
GPU Identified at: /device:GPU:0


If the above command could not identify your GPU then click runtime -> change runtime type -> gpu

**Mount google drive in order to pull in custom datasets or wav files. Skip if you plan to use wget instead.**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Download pretrained models to use for generation, just replace the wget links here. Alternatively could pull from gdrive**

256x256 horse: http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-horse-config-f.pkl

512x384 car: http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-car-config-f.pkl

1024x1024 ffhq: http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl

My models:

512x512 Materials: https://lucianet.s3.amazonaws.com/materials.pkl

In [23]:
# lets create a subfolder for the models to not make a mess... also makes it easy to save and reload them from gdrive if we get annoyed
model_dir = './models'
model_filename = 'materials.pkl'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
%cd models/
!wget https://lucianet.s3.amazonaws.com/materials.pkl
%cd ..

/content/stylegan2-ada/models
--2020-11-22 06:19:52--  https://lucianet.s3.amazonaws.com/materials.pkl
Resolving lucianet.s3.amazonaws.com (lucianet.s3.amazonaws.com)... 52.217.42.236
Connecting to lucianet.s3.amazonaws.com (lucianet.s3.amazonaws.com)|52.217.42.236|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 363968274 (347M) [application/x-www-form-urlencoded]
Saving to: ‘materials.pkl’

materials.pkl       100%[===================>] 347.11M  34.6MB/s    in 11s     

2020-11-22 06:20:03 (32.6 MB/s) - ‘materials.pkl’ saved [363968274/363968274]

/content/stylegan2-ada


**Load our music**

Similar to the models, will make a subfolder for music so that reinstantiation from gdrive is simple

In [7]:
music_dir = './wavs'
music_filename = 'losemymind.wav'
if not os.path.exists(music_dir):
    os.makedirs(music_dir)

In [5]:
# example wav here from wget
%cd wavs
!wget -o wavs/losemymind.wav https://lucianet.s3.amazonaws.com/losemymind.wav
%cd ..

In [8]:
# example from gdrive
!cp /content/drive/MyDrive/losemymind.wav ./wavs/losemymind.wav

**Extract features from our wav file**

In [24]:
# temporary installer so i can locate missing dependencies for a requirements.txt 
!pip install soundfile
!apt install ffmpeg

     |████████████████████████████████| 1.6MB 14.4MB/s 
  Created wheel for librosa: filename=librosa-0.6.3-cp36-none-any.whl size=1573314 sha256=a05cea2c99d35bbb6bd83d0c251f20242f7da934fdec7b4b4ad08a5ddc6954e6
  Stored in directory: /root/.cache/pip/wheels/ce/49/68/87ba660f30e3977f0778e39ee2e944629cd37c2a0ce41f9ff1
Successfully built librosa
  Found existing installation: librosa 0.6.2
    Uninstalling librosa-0.6.2:
      Successfully uninstalled librosa-0.6.2


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [9]:
# import feature extractor
from music import feature_extraction

In [10]:
feature_extraction.extract_features(f'{music_dir}/{music_filename}')

setting bpm 150
=== Processing low pass filter ===
=== Processing band pass filter ===
=== Processing high pass filter ===


**Generate frames from extracted features**

In [11]:
# import music generator
from music import music_generator

In [24]:
# length of influence
length_high=5
length_mid=3
length_low=16

# features -> layers of influence
lpf = [0, 1, 2]
bpf = [3, 4, 5]
hpf = [6, 7, 8, 9, 10, 11, 12, 13]

# generator from music
music_generator.generate_from_music(f'{model_dir}/{model_filename}',
                                    './features',
                                    length_high=length_high,
                                    length_mid=length_mid,
                                    length_low=length_low,
                                    lpf=lpf,
                                    bpf=bpf,
                                    hpf=hpf
                                    )

Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.

Gs                            Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
G_mapping/Normalize           -         (?, 512)            -               
G_mapping/Dense0              262656    (?, 512)            (512, 512)      
G_mapping/Dense1              262656    (?, 512)            (512, 512)      
G_mapping/Dense2              262656    (?, 512)            (512, 512)      
G_mapping/Dense3              262656    (?, 512)            (512, 512)      
G_mapping/Dense4              262656    (?, 512)            (512, 512)      
G_mapping/Dense5              262656    (?, 512)            (512, 512)   

100%|██████████| 9771/9771 [23:49<00:00,  6.83it/s]


**Generate video from frames**

In [25]:
# import music generator
from music import video_generator

In [33]:
fps=40
resolution=512
out_dir='./output'
out_file='generated.mp4'
video_generator.create_video(fps=fps,
                             resolution=resolution,
                             input_path='/content/stylegan2-ada/imgs/',
                             music_path=f'{music_dir}/{music_filename}',
                             out_dir=out_dir,
                             out_file=out_file)

100%|██████████| 9771/9771 [00:20<00:00, 472.21it/s]


In [46]:
# display video in notebook to download (much faster than downloading manually)
!pip install -U kora
from kora.drive import upload_public
url = upload_public(f'generated.mp4')
# then display it
from IPython.display import HTML
HTML(f"""<video src={url} width=500 height=500 controls/>""")

Requirement already up-to-date: kora in /usr/local/lib/python3.6/dist-packages (0.9.11)


^ video wont play, but click the 3 dots to download 